In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request
import os
import requests
import shutil
from tqdm import tqdm
import re
import pickle
import json

### scrap album info

In [29]:
all_albums = []
all_urls = []

for year in range(2002,2018) :
    
    url = "https://www.billboard.com/charts/year-end/"+str(year)+"/top-billboard-200-albums"
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib.request.urlopen(req)
    res = bs(html.read(),"html5lib")
    album_titles = [re.sub("\n", '', c.text) for c in res.findAll(attrs={'class': "ye-chart-item__title"})]
    album_artists = [re.sub("\n", '', c.text) for c in res.findAll(attrs={'class': "ye-chart-item__artist"})]
    urls = [c.find("img")["src"] for c in res.findAll(attrs={'class': "ye-chart-item__image"})]
    all_urls.extend(urls)
    
    print(len(album_titles), len(album_artists), len(urls))
    
    
    all_albums.extend([{'album': x[0], 
                        'artist' : x[1], 
                        'year' : year, 
                        'rank' : r + 1, 
                        'img_src' : "albums_covers/" + re.sub("\W+", "", x[2].split('/')[-1][:-4]) + ".jpg"} for r, x in enumerate(zip(album_titles, album_artists, album_image_src))])
    
all_albums[:10]

192 192 192
197 197 197
198 198 198
197 197 197
199 199 199
199 199 199
199 199 199
198 198 198
198 198 198
194 194 194
198 198 198
198 198 198
200 200 200
199 199 199
200 200 200
200 200 200


[{'album': 'The Eminem Show',
  'artist': 'Eminem',
  'year': 2002,
  'rank': 1,
  'img_src': 'albums_covers/kendricklamarh1y53x53.jpg'},
 {'album': 'Weathered',
  'artist': 'Creed',
  'year': 2002,
  'rank': 2,
  'img_src': 'albums_covers/brunomarsva753x53.jpg'},
 {'album': 'Nellyville',
  'artist': 'Nelly',
  'year': 2002,
  'rank': 3,
  'img_src': 'albums_covers/theweekndgvxstarboy6xr53x53.jpg'},
 {'album': 'M!ssundaztood',
  'artist': 'P!nk',
  'year': 2002,
  'rank': 4,
  'img_src': 'albums_covers/edsheeran3vg53x53.jpg'},
 {'album': '[Hybrid Theory]',
  'artist': 'Linkin Park',
  'year': 2002,
  'rank': 5,
  'img_src': 'albums_covers/drakezwl53x53.jpg'},
 {'album': 'O Brother, Where Art Thou?',
  'artist': 'Soundtrack',
  'year': 2002,
  'rank': 6,
  'img_src': 'albums_covers/soundtrack000moanayt053x53.jpg'},
 {'album': 'Silver Side Up',
  'artist': 'Nickelback',
  'year': 2002,
  'rank': 7,
  'img_src': 'albums_covers/postmalonetp6stoney8ge53x53.jpg'},
 {'album': 'Britney',
  'ar

### save info

In [30]:
with open('all_albums.json', 'w') as fp:
    json.dump(all_albums, fp)

### save album images

In [ ]:
urls = [c["img_src"] for c in all_albums]
urls[:3]

In [ ]:
if not os.path.exists("albums_covers"):
    os.makedirs("albums_covers")
    
for url in tqdm(urls) :
    r = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
    
    if r.status_code == 200:
        
        name = "albums_covers/" + re.sub("\W+", "", url.split('/')[-1][:-4]) + ".jpg"
                
        with open(name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)    